In [1]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam
from imutils import paths
import imagesize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import cv2
import os

2022-04-26 15:03:58.768512: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-26 15:03:58.768594: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class modelNet:
    @staticmethod
    def LeNet(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        
        model.add(Conv2D(4, (3, 3), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        
        # model.add(Conv2D(16, (5, 5), padding="same"))
        # model.add(Activation("relu"))
        # model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(10))
        model.add(Activation("relu"))
        
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model
    
    @staticmethod
    def VGG16(classes):
        # initializing the model
        model = Sequential()
        
        
        model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
        model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
        
        model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
        
        model.add(Flatten())
        model.add(Dense(units=4096,activation="relu"))
        model.add(Dense(units=4096,activation="relu"))
        
        model.add(Dense(units=classes, activation="softmax"))
    

In [3]:
EPOCHS = 10
INIT_LR = 1e-3
BS = 8
CLASSES = 3
WIDTH = 224
HEIGHT = 224

In [4]:
print("[INFO] loading images...")
data = []
labels = []

[INFO] loading images...


In [5]:
imagePaths = sorted(list(paths.list_images("augData/trialDataset")))
random.seed(42)
random.shuffle(imagePaths)

### Visualizing image Size

In [6]:
img_meta = {}
for path in imagePaths: img_meta[path] = imagesize.get(path)

# converting the sizes into graph
img_meta_df = pd.DataFrame.from_dict([img_meta]).T.reset_index().set_axis(['FileName', 'Size'], axis='columns', inplace=False)
img_meta_df[["Width", "Height"]] = pd.DataFrame(img_meta_df["Size"].tolist(), index=img_meta_df.index)
img_meta_df["Aspect Ratio"] = round(img_meta_df["Width"] / img_meta_df["Height"], 2)

print(f'Total Nr of Images in the dataset: {len(img_meta_df)}')
img_meta_df.mean(axis=0)


Total Nr of Images in the dataset: 2522


/tmp/ipykernel_16984/2647815427.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  img_meta_df.mean(axis=0)


Width           239.806899
Height          486.275971
Aspect Ratio      0.510218
dtype: float64

In [7]:
# Visualize Image Resolutions

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
points = ax.scatter(img_meta_df.Width, img_meta_df.Height, color='blue', alpha=0.5, s=img_meta_df["Aspect Ratio"]*100, picker=True)
ax.set_title("Image Resolution")
ax.set_xlabel("Width", size=14)
ax.set_ylabel("Height", size=14)
plt.savefig("img_plot.png")
del(img_meta_df)

### Taking out the images and training the model

In [8]:
for imagePath in imagePaths:    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (WIDTH, HEIGHT))
    image = img_to_array(image)
    data.append(image)
    

In [9]:
unique, counts = np.unique(np.array(labels), return_counts=True)
dict(zip(unique, counts))

{'double_person': 947, 'single_person': 963, 'triple_person': 612}

In [10]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [11]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)
del(data)
del(labels)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [12]:
print("[INFO] compiling model...")
model = modelNet.LeNet(width=WIDTH, height=HEIGHT, depth=3, classes=CLASSES)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


2022-04-26 15:04:58.020023: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-26 15:04:58.022600: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-26 15:04:58.024319: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (awi-stavya): /proc/driver/nvidia/version does not exist
2022-04-26 15:04:58.065949: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/hom

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 4)       112       
                                                                 
 activation (Activation)     (None, 224, 224, 4)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 4)      0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 dense (Dense)               (None, 10)                501770    
                                                                 
 activation_1 (Activation)   (None, 10)                0         
                                                        

In [14]:
print("[INFO] training network...")
H = model.fit(trainX, trainY, batch_size=BS,
    validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)
# save the model to disk
print("[INFO] serializing network...")
model.save("model_1.h5", save_format="h5")

[INFO] training network...


2022-04-26 15:05:03.416674: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1138593792 exceeds 10% of free system memory.


Epoch 1/10
236/236 [==============================] - ETA: 0s - loss: 1.1636 - accuracy: 0.3782

2022-04-26 15:05:49.000933: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 379932672 exceeds 10% of free system memory.


236/236 [==============================] - 48s 194ms/step - loss: 1.1636 - accuracy: 0.3782 - val_loss: 1.0865 - val_accuracy: 0.3788
Epoch 2/10
236/236 [==============================] - 42s 179ms/step - loss: 1.0870 - accuracy: 0.3829 - val_loss: 1.0796 - val_accuracy: 0.3788
Epoch 3/10
236/236 [==============================] - 43s 181ms/step - loss: 1.0846 - accuracy: 0.3818 - val_loss: 1.0762 - val_accuracy: 0.3788
Epoch 4/10
236/236 [==============================] - 41s 172ms/step - loss: 1.0829 - accuracy: 0.3813 - val_loss: 1.0746 - val_accuracy: 0.3788
Epoch 5/10
236/236 [==============================] - 42s 178ms/step - loss: 1.0819 - accuracy: 0.3861 - val_loss: 1.0736 - val_accuracy: 0.3788
Epoch 6/10
236/236 [==============================] - 35s 150ms/step - loss: 1.0817 - accuracy: 0.3829 - val_loss: 1.0728 - val_accuracy: 0.3788
Epoch 7/10
236/236 [==============================] - 35s 149ms/step - loss: 1.0824 - accuracy: 0.3808 - val_loss: 1.0724 - val_accuracy: 0.3

In [15]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Double and triple rider")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot_multi_class_4.png")

In [16]:
predictions = model.predict(x=testX, batch_size=BS)

print(predictions.shape)

2022-04-26 15:12:04.243892: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 379932672 exceeds 10% of free system memory.


(631, 3)


In [17]:
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1),target_names=lb.classes_))

               precision    recall  f1-score   support

double_person       0.00      0.00      0.00       251
single_person       0.38      1.00      0.55       239
triple_person       0.00      0.00      0.00       141

     accuracy                           0.38       631
    macro avg       0.13      0.33      0.18       631
 weighted avg       0.14      0.38      0.21       631



/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

### Trying out test set with 3 classification

In [18]:
test_imagepaths = sorted(list(paths.list_images("testSet")))
random.seed(42)
random.shuffle(test_imagepaths)

In [19]:
test_labels = []
test_data = []
for imagePath in test_imagepaths:    
    label = imagePath.split(os.path.sep)[-2]
    test_labels.append(label)
    
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (WIDTH, HEIGHT))
    image = img_to_array(image)
    test_data.append(image)

test_labels = np.array(test_labels)
test_data = np.array(test_data, dtype="float") / 255.0


In [20]:
lb = LabelBinarizer()
test_labels = lb.fit_transform(test_labels)

In [21]:
test_predictions = model.predict(x=test_data, batch_size=32)

print(test_predictions.shape)

(221, 3)


In [22]:
test_predictions.argmax(axis=1)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1])

In [23]:
print(classification_report(test_labels.argmax(axis=1),test_predictions.argmax(axis=1),target_names=lb.classes_))

               precision    recall  f1-score   support

double_person       0.00      0.00      0.00        78
single_person       0.36      1.00      0.53        80
triple_person       0.00      0.00      0.00        63

     accuracy                           0.36       221
    macro avg       0.12      0.33      0.18       221
 weighted avg       0.13      0.36      0.19       221



/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/awiros-tech/anaconda3/envs/tripleRider/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,